In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Notebook detailing approach of moving from Tensorflow to Tensorflow.js

Original network based on Microsoft Human Pose Estimation (https://github.com/microsoft/human-pose-estimation.pytorch) adopted for Tensorflow v2.2.

* Note this isn't a full notebook, it's here purely to demonstrate how to migrate a model to TF.js

In [34]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import layers

from easydict import EasyDict as edict

In [37]:
import tensorflowjs as tfjs
from model import get_pose_net

### Config for model, etc.

* Handle pretrained = true

In [48]:
cfg = edict({
    'MODEL': {
        'IMAGE_SIZE': [256, 256],
        'NUM_JOINTS': 11,
        'INIT_WEIGHTS': False,
        'EXTRA': {
            'HEATMAP_SIZE': [64, 64],
            'DECONV_WITH_BIAS': False,
            'FINAL_CONV_KERNEL': 1,
            'NUM_DECONV_KERNELS': [4, 4, 4],
            'NUM_DECONV_FILTERS': [256, 256, 256]
        }
}})

### Create model

Note that:

1. the original Microsoft paper has custom loss functions, these have been replaced here purely to simplify the migration.
1. For training a model, the custom loss functions from the paper and repo should be used.
1. We are currently not loading a trained model here (fix later!)

In [45]:
model = get_pose_net(cfg, is_train=False)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [47]:
model.summary()

Model: "pose_mobilenetv2_1.00"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 257, 257, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32) 128         Conv1[0][0]                      
______________________________________________________________________________

### Convert model

From https://www.tensorflow.org/js/tutorials/conversion/import_keras

In [51]:
tfjs.converters.save_keras_model(model, './app/models/')
